# Predicting the prevalence of Diabetes from online records using 

The main goal of this assignment is to set up a machine learning framework that predicts the prevalence of a specific 
chronic disease or health risk factor from digital records of online behavior.



## Ground truth data

### Downloading the dataset

In [ ]:
https://chronicdata.cdc.gov/resource/5hnu-2c66.json

In [1]:
import pandas as pd

df = pd.read_csv("data/U.S._Chronic_Disease_Indicators__CDI_.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (7,10,18,19,20,21,23,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(5)

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2015,2015,AK,Alaska,YRBSS,Alcohol,Alcohol use among youth,NaN,%,Crude Prevalence,...,2,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN
1,2015,2015,AL,Alabama,YRBSS,Alcohol,Alcohol use among youth,NaN,%,Crude Prevalence,...,1,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2015,2015,AR,Arkansas,YRBSS,Alcohol,Alcohol use among youth,NaN,%,Crude Prevalence,...,5,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2015,2015,AZ,Arizona,YRBSS,Alcohol,Alcohol use among youth,NaN,%,Crude Prevalence,...,4,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2015,2015,CA,California,YRBSS,Alcohol,Alcohol use among youth,NaN,%,Crude Prevalence,...,6,ALC,ALC1_1,CrdPrev,OVERALL,OVR,NaN,NaN,NaN,NaN


In [2]:
unique_topics = list(set(df["Topic"]))
unique_topics

['Cancer',
 'Diabetes',
 'Chronic Kidney Disease',
 'Cardiovascular Disease',
 'Overarching Conditions',
 'Chronic Obstructive Pulmonary Disease',
 'Immunization',
 'Reproductive Health',
 'Oral Health',
 'Older Adults',
 'Disability',
 'Asthma',
 'Arthritis',
 'Alcohol',
 'Mental Health',
 'Nutrition, Physical Activity, and Weight Status',
 'Tobacco']

In [3]:
df_diabetes = df[df.Topic=="Diabetes"]
df_diabetes.to_csv("data/US_diabetes.csv")

In [4]:
set(df_diabetes.DataSource)

{'BRFSS', 'Birth Certificate, NVSS', 'NVSS', 'PRAMS', 'State Inpatient Data'}

### Type of questions with prevalence in name

In [24]:
questions = dict(enumerate(list(set(df_diabetes.Question))))
questions

{0: 'Prevalence of diagnosed diabetes among adults aged >= 18 years',
 1: 'Pneumococcal vaccination among noninstitutionalized adults aged >= 65 years with diagnosed diabetes',
 2: 'Mortality with diabetic ketoacidosis reported as any listed cause of death',
 3: 'Amputation of a lower extremity attributable to diabetes',
 4: 'Diabetes prevalence among women aged 18-44 years',
 5: 'Prevalence of depressive disorders among adults aged >= 18 years with diagnosed diabetes',
 6: 'Dilated eye examination among adults aged >= 18 years with diagnosed diabetes',
 7: 'Pneumococcal vaccination among noninstitutionalized adults aged 18-64 years with diagnosed diabetes',
 8: 'Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes',
 9: 'Mortality due to diabetes reported as any listed cause of death',
 10: 'Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes',
 11: 'Adults with diagnosed diabetes aged >= 18 years who have taken 

## Type of Data Values


In [6]:
data_values = set(df_diabetes.DataValueType)
data_values

{'Age-adjusted Prevalence',
 'Age-adjusted Rate',
 'Crude Prevalence',
 'Crude Rate',
 'Number',
 'Prevalence'}

In [40]:
def get_prevalence_indicator(df, state, category, year, data_value, question):
    return float(df[
        (df.LocationDesc == state) & \
        (df.StratificationCategory1 == category) & \
        (df.YearStart == year) & \
        (df.DataValueType == data_value) & \
        (df.Question == question )
    ]["DataValue"].values[0])

In [41]:
get_prevalence_indicator(df=df_diabetes,
                         state="Wyoming",
                         category="Overall",
                         year=2013,
                         data_value="Age-adjusted Prevalence",
                         question = questions[0]                        
                        )

8.1

In [32]:
get_prevalence_indicator(df=df_diabetes,
                         state="Wyoming",
                         year=2015,
                         data_value="Crude Prevalence")

,Question,DataValue
34470,Adults with diagnosed diabetes aged >= 18 year...,51.8
35348,Prevalence of high cholesterol among adults ag...,66.2
36227,Prevalence of high blood pressure among adults...,71.8
37105,Prevalence of depressive disorders among adult...,28
37984,Influenza vaccination among noninstitutionaliz...,41.8
38866,Influenza vaccination among noninstitutionaliz...,60
39746,Pneumococcal vaccination among noninstitutiona...,58.1
40622,Pneumococcal vaccination among noninstitutiona...,84.1
41456,Prevalence of diagnosed diabetes among adults ...,8.4
41779,Diabetes prevalence among women aged 18-44 years,3.7


In [10]:
get_prevalence_indicator(df=df_diabetes,
                         state="Wyoming",
                         year=2015,
                         data_value="Prevalence")

,Question,DataValue


In [13]:
df_diabetes[df_diabetes.DataValueType == "Prevalence"].to_csv("prevalence_states.csv")